In [77]:
!pip install nbformat nbconvert


[notice] A new release of pip available: 22.3 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [78]:
import mysql.connector
from mysql.connector import Error
import os
from dotenv import load_dotenv
import pandas as pd
from sqlalchemy import create_engine, inspect


In [79]:
import nbformat
from nbconvert.preprocessors import ExecutePreprocessor

# Ruta al notebook que deseas ejecutar
notebook_path = './3_delete_tables_in_aiven.ipynb'

def ejecutar_notebook(notebook_path):
    try:
        # Leer el notebook con codificación utf-8
        with open(notebook_path, encoding='utf-8') as f:
            nb = nbformat.read(f, as_version=4)

        # Configurar el preprocesador para ejecutar el notebook
        ep = ExecutePreprocessor(timeout=600, kernel_name='python3')

        # Ejecutar el notebook
        ep.preprocess(nb, {'metadata': {'path': './'}})
        print(f'✅ El notebook {notebook_path} se ha ejecutado correctamente.')

    except UnicodeDecodeError as e:
        print(f"❌ Error de decodificación en {notebook_path}: {e}")
    except Exception as e:
        print(f'❌ Error al ejecutar el notebook {notebook_path}: {e}')


ejecutar_notebook(notebook_path)
    

✅ El notebook ./3_delete_tables_in_aiven.ipynb se ha ejecutado correctamente.


In [80]:
# Cargar variables de entorno desde el archivo .env
load_dotenv()

# Obtener las variables de entorno
host = os.getenv('host')
port = os.getenv('port')
database = os.getenv('database')
user = os.getenv('user')
password = os.getenv('password')


In [81]:
# csv_file_path = '../data/data_processed/MunE19_c_mod_municipios.csv'
dir_path = '../data/data_processed/'

In [82]:
# table_name = 'municipios_pruebas_3'
# primary_key_column = 'index'

In [83]:
# try:
#     # Connect to the MySQL database
#     connection = mysql.connector.connect(
#         user=user,
#         password=password,
#         host=host,
#         port=port,
#         database=database
#     )
    
#     if connection.is_connected():
#         print("✅ Successfully connected to the database")

#         df = pd.read_csv(csv_file_path, sep=',', encoding='utf-8')
#         print(df.head())
#         print(df.info())

#         # Handle NaN values in the DataFrame
#         df = df.where(pd.notnull(df), None)  # Replace NaN with None (NULL in MySQL)

#         # Create a cursor and execute the query
#         cursor = connection.cursor()

#         # Create table if it does not exist
#         # Generate a CREATE TABLE statement based on the DataFrame columns
#         columns = ', '.join(f"`{col}` VARCHAR(255)" for col in df.columns)
#         create_table_query = f"CREATE TABLE IF NOT EXISTS `{table_name}` ({columns}, PRIMARY KEY (`{primary_key_column}`))"
#         cursor.execute(create_table_query)
#         print(f"🛠️ Table '{table_name}' created or already exists.")

#         # Insert data into the table
#         try:
#             print(f"ℹ️ Inserting data into '{table_name}'...")

#             # Iterate over each row in the DataFrame
#             for i, row in enumerate(df.itertuples(index=False), 1):
#                 # Generate the INSERT INTO query with properly escaped column names
#                 insert_query = f"INSERT INTO `{table_name}` (`{'`, `'.join(df.columns)}`) VALUES ({', '.join(['%s']*len(df.columns))})"
                
#                 # Execute the query with the values from the current row
#                 cursor.execute(insert_query, tuple(row))
                
#                 # Print progress message
#                 if i % 10 == 0:
#                     print(f"Inserted {i} rows...")

#             # Commit the changes
#             connection.commit()
#             print(f"✨ Data successfully inserted into '{table_name}'")

#         except Error as e:
#             # Handle any errors that may occur during insertion
#             print("❌ Error inserting data:", e)

# except Error as e:
#     print("❌ An error occurred:", e)

# finally:
#     if connection.is_connected():
#         cursor.close()
#         connection.close()
#         print("🔒 MySQL connection is closed")

In [84]:
def limpiar_nombre(nombre):
    return os.path.splitext(nombre)[0]

In [85]:
# Lista para almacenar el estado de cada tabla
resumen = []


In [86]:
try:
    # Conectar a la base de datos MySQL
    connection = mysql.connector.connect(
        user=user,
        password=password,
        host=host,
        port=port,
        database=database
    )

    if connection.is_connected():
        print("✅ Conectado a la base de datos")

        for archivo in os.listdir(dir_path):
            if archivo.endswith('.csv'):
                # Ruta completa al archivo CSV
                ruta_archivo = os.path.join(dir_path, archivo)
                
                try:
                    # Lee el archivo CSV con pandas
                    df = pd.read_csv(ruta_archivo, sep=',', encoding='utf-8')
                    print(f"\n\n📄 Procesando archivo: {archivo}")
                    print(df.head())
                    print(df.info())

                    # Limpia el nombre del archivo para usarlo como nombre de la tabla
                    table_name = limpiar_nombre(archivo)

                    # Manejar valores NaN en el DataFrame
                    df = df.where(pd.notnull(df), None)  # Reemplazar NaN con None (NULL en MySQL)

                    # Eliminar columna 'index' si existe
                    # if 'index' in df.columns:
                    #     df.drop(columns=['index'], inplace=True)

                    # Crear un cursor y ejecutar la consulta
                    cursor = connection.cursor()

                    # Crear la tabla si no existe
                    columns = ', '.join(f"`{col}` VARCHAR(255)" for col in df.columns)
                    # primary_key_column = df.columns[0]  # Asumir que la primera columna es la clave primaria
                    primary_key_column = 'index'
                    create_table_query = f"CREATE TABLE IF NOT EXISTS `{table_name}` ({columns}, PRIMARY KEY (`{primary_key_column}`))"
                    cursor.execute(create_table_query)
                    print(f"🛠️ Tabla '{table_name}' creada o ya existe.")

                    # Insertar datos en la tabla
                    try:
                        print(f"ℹ️ Insertando datos en '{table_name}'...")

                        for i, row in enumerate(df.itertuples(index=False), 1):
                            insert_query = f"INSERT INTO `{table_name}` (`{'`, `'.join(df.columns)}`) VALUES ({', '.join(['%s']*len(df.columns))})"
                            cursor.execute(insert_query, tuple(row))

                            if i % 10 == 0:
                                print(f"Insertadas {i} filas...")

                        connection.commit()
                        print(f"✨ Datos insertados exitosamente en '{table_name}'")
                        resumen.append((table_name, '✅'))

                    except Error as e:
                        print("❌ Error al insertar datos:", e)
                        resumen.append((table_name, f"❌ Error al insertar datos: {e}"))

                except Error as e:
                    print(f"❌ Error procesando el archivo {archivo}: ", e)
                    resumen.append((table_name, f"❌ Error procesando el archivo: {e}"))

finally:
    if connection.is_connected():
        cursor.close()
        connection.close()
        print("🔒 Conexión a MySQL cerrada")

# Imprimir resumen
print("\nResumen de la ejecución:")
for tabla, estado in resumen:
    print(f"Tabla: {tabla} - Estado: {estado}")

✅ Conectado a la base de datos


📄 Procesando archivo: MunE19_c_mod_municipios.csv
   index           TH  Cod Comarca  Cod Municipio               Ámbito
0      0  Araba-Alava            1             59      VITORIA-GASTEIZ
1      1  Araba-Alava            2              1     ALEGRÍA-DULANTZI
2      2  Araba-Alava            2              3              ARAMAIO
3      3  Araba-Alava            2              6              ARMIÑÓN
4      4  Araba-Alava            2              8  ARRATZUA-UBARRUNDIA
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 254 entries, 0 to 253
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   index          254 non-null    int64 
 1   TH             254 non-null    object
 2   Cod Comarca    254 non-null    int64 
 3   Cod Municipio  254 non-null    int64 
 4   Ámbito         254 non-null    object
dtypes: int64(3), object(2)
memory usage: 10.0+ KB
None
🛠️ Tabla 'MunE19_c_mod_municip

In [87]:
# connection_str = f"mysql+mysqlconnector://{user}:{password}@{host}:{port}/{database}"

# try:
#     # Conectar a la base de datos MySQL con SQLAlchemy
#     engine = create_engine(connection_str)
#     connection = engine.connect()

#     print("✅ Conexión exitosa a la base de datos")

#     # Consultar los datos de la tabla y cargarlos en un DataFrame usando SQLAlchemy
#     query = f"SELECT * FROM `{table_name}`"
#     df = pd.read_sql(query, con=engine)

#     # Mostrar los primeros 5 registros para verificar
#     print(f"🔍 Tabla '{table_name}':")
#     print(df.head())
#     print(df.info())

# except mysql.connector.Error as e:
#     print(f"❌ Error al conectar a la base de datos: {e}")

# finally:
#     # Cerrar la conexión a la base de datos
#     if 'connection' in locals() and connection is not None:
#         connection.close()
#         print("🔒 Conexión MySQL cerrada")

In [88]:


# Define la cadena de conexión a la base de datos
connection_str = f"mysql+mysqlconnector://{user}:{password}@{host}:{port}/{database}"

try:
    # Conectar a la base de datos MySQL con SQLAlchemy
    engine = create_engine(connection_str)
    connection = engine.connect()

    print("✅ Conexión exitosa a la base de datos")

    # Crear un inspector para obtener información de la base de datos
    inspector = inspect(engine)

    # Obtener la lista de todas las tablas en la base de datos
    tablas = inspector.get_table_names()
    print(f"🔍 Tablas encontradas: {tablas}")

    # Iterar sobre cada tabla y consultar su contenido
    for table_name in tablas:
        try:
            query = f"SELECT * FROM `{table_name}`"
            df = pd.read_sql(query, con=engine)

            # Mostrar los primeros 5 registros para verificar
            print(f"\n🔍 Tabla '{table_name}':")
            print(df.head())
            print(df.info())
        except Exception as e:
            print(f"❌ Error al consultar la tabla '{table_name}': {e}")

except Exception as e:
    print(f"❌ Error al conectar a la base de datos: {e}")

finally:
    # Cerrar la conexión a la base de datos
    if 'connection' in locals() and connection is not None:
        connection.close()
        print("🔒 Conexión MySQL cerrada")

✅ Conexión exitosa a la base de datos
🔍 Tablas encontradas: ['MunE19_c_mod_municipios', 'MunE19_c_mod_resultados_municipios', 'MunE19_c_mod_votos_partido_municipios']

🔍 Tabla 'MunE19_c_mod_municipios':
  index           TH Cod Comarca Cod Municipio            Ámbito
0     0  Araba-Alava           1            59   VITORIA-GASTEIZ
1     1  Araba-Alava           2             1  ALEGRÍA-DULANTZI
2    10  Araba-Alava           2            17   CAMPEZO/KANPEZU
3   100      Bizkaia           3            91           ATXONDO
4   101      Bizkaia           3            92             BEDIA
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 254 entries, 0 to 253
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   index          254 non-null    object
 1   TH             254 non-null    object
 2   Cod Comarca    254 non-null    object
 3   Cod Municipio  254 non-null    object
 4   Ámbito         254 non-null    object
dt